In [152]:
from tatc.analysis.latency import collect_downlinks, aggregate_downlinks, compute_latency
from tatc.analysis.coverage import collect_observations
from tatc.schemas.point import GroundStation
from tatc.schemas.satellite import Satellite
from tatc.schemas.instrument import Instrument
from tatc.schemas.orbit import TwoLineElements
from tatc.schemas.point import Point
from tatc.analysis.track import collect_ground_track
from datetime import datetime, timedelta
import geopandas as gpd
from Storm_analysis.geos_obs import get_observations
import contextily as ctx
import geoplot as gplt
from datetime import timezone
import datetime as dt
import pandas as pd
import time


In [153]:
gr1=GroundStation(name= "Alaska",
   latitude = 64.859,
   longitude = -147.854,
   min_elevation_angle = 25,
   min_access_time= 10)

In [154]:
ins=Instrument(name= "Scon1",
     field_of_regard= 20,
     min_access_time= 10,
     duty_cycle= 0.3,
     duty_cycle_scheme= "fixed")
po1= Point(id= 1959,
   latitude= 30.510237406115554,
   longitude= -97.70977402414428)

In [155]:
orb=TwoLineElements(type= "tle",
    tle= ["1 27424U 02022A   22167.05119303  .00000638  00000+0  15103-3 0  9994",
"2 27424  98.2477 108.9546 0000928  92.9194 327.0802 14.57300770 69982"])

In [156]:
sat1=Satellite(type= "satellite",
   name= "ISS-test",
   orbit= orb,
   instruments= [ins])
instrument={
    "name": ins.name,
    "field_of_regard": ins.field_of_regard,
    "min_access_time": ins.min_access_time,
    "req_self_sunlit": None,
    "req_target_sunlit": None
}
test_sat={
    "type": "satellite",
    "name": sat1.name,
    "orbit": {
        "type":"tle",
        "tle":sat1.orbit.tle
    },
    "instrument": instrument
}

In [157]:
start= datetime.fromisoformat("2022-05-22T00:00:00+00:00")
end= datetime.fromisoformat("2022-05-22T04:00:00+00:00")
utc = dt.timezone.utc
start= dt.datetime(2022,5,22,0,0, tzinfo= utc)
end= dt.datetime(2022,5,22, 4,0, tzinfo=utc)
obs_delta= timedelta(minutes=30)

In [164]:
gdf,storms= get_observations(test_sat,start, start+obs_delta, instrument)
#storms=storms.reset_index()
times = pd.date_range(
        start = start,
        end=start+obs_delta,
        freq = dt.timedelta(seconds = 300)
    ).tz_convert(timezone.utc)
g_track=collect_ground_track(sat1, ins, times, mask=None)
g_track.plot()
storms.plot(ax=g_track.plot())
gdf.plot()

ValueError: too many values to unpack (expected 2)

In [150]:
t_tim=start
while t_tim<end:
    sim_end= t_tim + obs_delta
    #print("Observation time start_time", t_tim,"end", sim_end)
    #obs=collect_observations(po1, sat1, ins, t_tim, sim_end)
    t_tim= sim_end+ obs_delta
    #print('Downlink time start_time', sim_end, "end", t_tim)
    #downlinks= collect_downlinks(gr1, sat1,sim_end, t_tim)
    #down=False 
    #while not down:
    #    downlinks=collect_downlinks(gr1, sat1,t_tim, sim_end)
    
    
#df=collect_downlinks(gr1, sat1,start, end)

#ax1=gplt.pointplot(g_track,
#                  color='brown',
#                   extent=(-180,-90,180,90),
#                  projection= gplt.PlateCarree()
#                  )
#ax=gplt.pointplot(storms,
#                  ax=ax1,
#                 extent=(-180,-90,180,90),
#                 projection= gplt.PlateCarree())
#ctx.add_basemap(ax,
#               source=ctx.providers.Stamen.TerrainBackground,
#               crs="epsg:4326",
#               attribution=False)
data=[]
storms
ico=10
for i in range(len(storms.geometry)):
    obs=collect_observations(Point(id= ico*i, latitude= storms.lat[i], longitude= storms.lon[i]), 
                             sat1, ins, start, start+obs_delta)
    
    data.append({'id': obs.point_id[0],
                 'geometry': obs.geometry[0],
                 'start': obs.start[0],
                 'end': obs.end[0],
                 'observed': obs.epoch[0]})

KeyError: 0

In [151]:
storms

,index,lat,lon,geometry,cnprcp,id,observation
0,42,-79.834740,-108.612503,POINT (-108.61250 -79.83474),0.0,42_20060522_0030,True
1,46,-79.447914,-113.115120,POINT (-113.11512 -79.44791),0.0,46_20060522_0030,True
2,47,-79.133621,-109.690735,POINT (-109.69073 -79.13362),0.0,47_20060522_0030,True
3,49,-78.500000,-85.343750,POINT (-85.34375 -78.50000),0.0,49_20060522_0030,True
4,55,-77.000000,-31.031250,POINT (-31.03125 -77.00000),0.0,55_20060522_0030,True
5,78,-71.187500,-134.500000,POINT (-134.50000 -71.18750),0.0,78_20060522_0030,True
6,79,-71.062500,-134.968750,POINT (-134.96875 -71.06250),0.0,79_20060522_0030,True
7,84,-70.687500,-137.343750,POINT (-137.34375 -70.68750),0.0,84_20060522_0030,True
8,85,-70.312500,-137.375000,POINT (-137.37500 -70.31250),0.0,85_20060522_0030,True
9,86,-69.750000,-135.531250,POINT (-135.53125 -69.75000),0.0,86_20060522_0030,True


In [85]:
obs=collect_observations(po1, sat1, ins, start, end)

In [88]:
obs

,point_id,geometry,satellite,instrument,start,end,epoch,sat_alt,sat_az,access,revisit
0,1959,POINT (-97.70977 30.51024),ISS-test,Scon1,2021-01-04 08:24:11.661677+00:00,2021-01-04 08:24:35.669996+00:00,2021-01-04 08:24:23.612246+00:00,81.531559,283.169719,0 days 00:00:24.008319,NaT
1,1959,POINT (-97.70977 30.51024),ISS-test,Scon1,2021-01-09 19:45:13.643821+00:00,2021-01-09 19:45:38.093980+00:00,2021-01-09 19:45:25.701652+00:00,81.623429,257.565916,0 days 00:00:24.450159,5 days 11:21:26.432303
2,1959,POINT (-97.70977 30.51024),ISS-test,Scon1,2021-01-13 08:22:35.196741+00:00,2021-01-13 08:23:08.022231+00:00,2021-01-13 08:22:51.658289+00:00,85.015500,282.581608,0 days 00:00:32.825490,3 days 12:37:54.378410
3,1959,POINT (-97.70977 30.51024),ISS-test,Scon1,2021-01-18 19:43:33.375566+00:00,2021-01-18 19:44:06.484097+00:00,2021-01-18 19:43:49.739630+00:00,85.261328,257.126262,0 days 00:00:33.108531,5 days 11:21:31.287356


In [89]:
results = gpd.GeoDataFrame()
downlinks = df
observations=obs
for _, observation in observations.iterrows():
    results = results.append(compute_latency(
                                observation,
                                    downlinks
                                    )
                                )

C:\Users\josue\AppData\Local\Temp\ipykernel_26504\1628359302.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(compute_latency(
C:\Users\josue\AppData\Local\Temp\ipykernel_26504\1628359302.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(compute_latency(
C:\Users\josue\AppData\Local\Temp\ipykernel_26504\1628359302.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(compute_latency(
C:\Users\josue\AppData\Local\Temp\ipykernel_26504\1628359302.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(compute_latency(


In [90]:
results

,point_id,geometry,satellite,observing_instrument,station_name,observed,downlinked,latency
0,1959,POINT (-97.70977 30.51024),ISS-test,Scon1,Alaska,2021-01-04 08:24:23.612246+00:00,2021-01-04 11:33:02.051639+00:00,0 days 03:08:38.439393
0,1959,POINT (-97.70977 30.51024),ISS-test,Scon1,Alaska,2021-01-09 19:45:25.701652+00:00,2021-01-09 21:34:31.919454+00:00,0 days 01:49:06.217802
0,1959,POINT (-97.70977 30.51024),ISS-test,Scon1,Alaska,2021-01-13 08:22:51.658289+00:00,2021-01-13 11:31:28.984310+00:00,0 days 03:08:37.326021
0,1959,POINT (-97.70977 30.51024),ISS-test,Scon1,Alaska,2021-01-18 19:43:49.739630+00:00,2021-01-18 21:32:56.180122+00:00,0 days 01:49:06.440492


In [92]:
a=observations.iterrows()

In [93]:
a

<generator object DataFrame.iterrows at 0x000002844A10D4A0>